In [1]:
# 参数管理: 我们可以通过一些特殊的操作对每层当中的网络当中的参数进行操作
# 包括初始化, 访问参数: 用于调试, 诊断和可视化. 以及生成共享层


In [2]:
# 我们先来看一些具有单隐藏层的多层感知机.

import torch
from torch import nn

# create Sequential: a sigual MLP (multilayer perceptron )
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
output = net(X)

output

tensor([[-0.6978],
        [-0.5541]], grad_fn=<AddmmBackward0>)

In [3]:
# 参数访问:
# 我们可以将Sequential看作一个列表容器, 我们可以通过索引看到其中的各个层的值
print(net[0]) # 取出第一层
# Linear(in_features=4, out_features=8, bias=True)

# 通过state_dict()返回这个层的参数. 
print(net[2].state_dict())
# 返回OrderDict这个数据类型.


Linear(in_features=4, out_features=8, bias=True)
OrderedDict([('weight', tensor([[-0.1413,  0.0627, -0.1605, -0.2589, -0.0890, -0.0094, -0.2776, -0.3399]])), ('bias', tensor([-0.1872]))])


In [4]:
# 目标参数, 
# 注意, 其实每个参数都表示为参数类的一个实例, 在torch中就是Parameter类
print(type(net[2].bias))
print(net[2].bias) # 打印bias的属性, 其中包括data, grad两个方面
dir(net[2].bias) # 查看bias的attribute
# 通过data取出数据: 
print(net[2].bias.data) # 就是单独的data
# 通过grad取出梯度
print(net[2].bias.grad) # None, 因为我们还没backward所以还没有计算梯度. 

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1872], requires_grad=True)
tensor([-0.1872])
None


In [5]:
# 访问梯度
net[2].weight.grad == None # True
# 因为我们还没有进行backward所以还没有计算梯度, 所以grad为None


True

In [6]:
# 一次性访问所有参数:
print(net.named_parameters()) # <generator object Module.named_parameters at 0x7fabcee9d7e0>
# 是一个生成器对象, 通过yield返回数值. 


# 访问net[0]的所有参数:
print(*[(name, parm) for name, parm in net[0].named_parameters()])

print("="*50)
# 访问net中的所有层, 所有参数:
print(*[(name, parm) for name, parm in net.named_parameters()]) # 返回net每层中参数 
# *进行解包. 


<generator object Module.named_parameters at 0x7fdacfbe8740>
('weight', Parameter containing:
tensor([[-0.3790,  0.3903,  0.3177, -0.3635],
        [ 0.2969,  0.1255,  0.1643,  0.3068],
        [-0.1398,  0.0216,  0.4197, -0.3163],
        [-0.2209,  0.1267,  0.1261, -0.3238],
        [ 0.1069, -0.3582,  0.1677,  0.0526],
        [ 0.4365, -0.2825, -0.1169,  0.1730],
        [ 0.0491, -0.2142, -0.2083,  0.0711],
        [-0.4516,  0.3154,  0.4188,  0.3988]], requires_grad=True)) ('bias', Parameter containing:
tensor([-0.2954, -0.4296,  0.4109,  0.0995, -0.4806, -0.4591,  0.2232,  0.2851],
       requires_grad=True))
('0.weight', Parameter containing:
tensor([[-0.3790,  0.3903,  0.3177, -0.3635],
        [ 0.2969,  0.1255,  0.1643,  0.3068],
        [-0.1398,  0.0216,  0.4197, -0.3163],
        [-0.2209,  0.1267,  0.1261, -0.3238],
        [ 0.1069, -0.3582,  0.1677,  0.0526],
        [ 0.4365, -0.2825, -0.1169,  0.1730],
        [ 0.0491, -0.2142, -0.2083,  0.0711],
        [-0.4516,  

In [7]:
# 因为state_dict()返回的本质是一个字典形式: 所以我们还可以通过key的形式进行访问参数
net.state_dict()['2.bias'].data # 通过key来查找

tensor([-0.1872])

In [8]:

# 现在让我们看看通过add_module添加网络层, 这样我们可以指定每层的名称

# 我们先定义black1:


def black1() -> nn.Sequential:
    return nn.Sequential(
        nn.Linear(4, 8), nn.ReLU(),
        nn.Linear(8, 4), nn.ReLU()
    )

# 然后定义black2: 
def black2() -> nn.Module:
    net = nn.Sequential() # definite a container 
    
    for i in range(4):
        # 嵌套
        net.add_module(f"black: {i}", black1())
        # 和我们直接在Sequential中定义的唯一区别, 就是这里我们可以执行模型的名字
    return net

rgnet = nn.Sequential(black2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2720],
        [0.2720]], grad_fn=<AddmmBackward0>)

In [9]:
# 打印网络结构:
print(rgnet)

Sequential(
  (0): Sequential(
    (black: 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (black: 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (black: 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (black: 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [10]:
# 由于是层是分层嵌套的, 所以我们也可以通过嵌套列表索引一样访问他们, 
rgnet[0][1][0].bias.data

tensor([ 0.4241, -0.1672, -0.1137,  0.2406, -0.0031,  0.2820, -0.1168, -0.2872])

In [26]:

# 我们使用的初始化函数, 都在torch.nn.init.*中, 
# 一般是: nn.init.***
# 参数初始化: 我们如何修改默认的函数, 来获得我们新的参数初始化
def init_norm(m: nn.Module):
    if type(m) == nn.Linear: # 如果是全连接层
        nn.init.normal_(m.weight,  mean=0, std=0.01) # 对w进行正态分布初始化
        nn.init.zeros_(m.bias) # 将bias设置为0 
        # _表示原地操作. 直接在原来的内存上进行操作

# nn.Module提供了一种方式就是可以递归遍历Module中的所有网络层. 
net.apply(init_norm) # 传入函数句柄, 然后apply即可. 

net[0].weight.data, net[0].bias.data[0]



(tensor([[ 0.0008, -0.0061, -0.0027, -0.0110],
         [-0.0191, -0.0167, -0.0239, -0.0059],
         [-0.0044, -0.0226,  0.0338, -0.0005],
         [-0.0152, -0.0104, -0.0025, -0.0038],
         [-0.0014, -0.0142, -0.0046,  0.0094],
         [ 0.0047, -0.0020, -0.0060, -0.0052],
         [-0.0058, -0.0047,  0.0162,  0.0053],
         [ 0.0001, -0.0200, -0.0034, -0.0198]]),
 tensor(0.))

In [36]:
# 当然我们还可以将所有参数初始化为给定的参数: 例如初始化为1:
def init_constant(m : nn.Module):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

# 在api我们可以将模型参数初始化为一个常量, 但是我们在实际的算法中是不被允许的.
# 为什么? 因为神经网络本质上是不断搜索参数的过程, 那么我们需要参数随机化, 然后不同的向着负梯度方向更新
# 但是如果将weight更新为同一个常量的话, 那么就导致我们的神经网络不起作用. 并且梯度下降算法难以起到作用
# 导致我们的效果变差, 并且weight难以被更新

net.apply(init_constant)

net[0].weight.data, net[0].bias.data
# 可见我们的模型参数就都被更新到了. 


        

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [45]:
# 我们还可以对某些块应用不同的初始化方法: 
# 例如我们使用Xavier方法更新第一层, 然后将第三个层初始化常量. 

# 首先实现一下我们初始化的函数: 
def init_xavier(m: nn.Module):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        # xavier有两种形式, 一种是uniform, 一种是norm
        nn.init.zeros_(m.bias)
def init_42(m: nn.Module):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) 
        # 为什么是42呢, 因为这是宇宙的答案
        # not panic!
# 因为net中的子层也是nn.Module. 所以我们同样可以针对某一层进行.apply
net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].weight.data)
print(net[2].weight.data) # 可见我们的数据都已经被格式化成为42了. 



tensor([[-0.2164,  0.1978, -0.0631, -0.4943],
        [ 0.6784,  0.6817,  0.5390,  0.0096],
        [-0.1690, -0.4595, -0.6092,  0.4800],
        [ 0.1883, -0.3058, -0.4731,  0.4180],
        [ 0.6634, -0.4068,  0.5876, -0.5340],
        [ 0.4437,  0.1796,  0.1938,  0.0240],
        [-0.6116, -0.0047,  0.4692, -0.5452],
        [-0.3827, -0.1719,  0.4193,  0.0935]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [47]:
# 自定义初始化, 我们可以使用我们自己的初始化函数, 而不必使用nn.init中的初始化方法. 
# 我们可以定义我们自己的初始化函数

def my_init(m):
    if type(m) == nn.Linear:
        print("init:", *[(name, param.shape) for name, param in m.named_parameters()[0]])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        # 保留绝对值 >= 5的那些权重

net.apply(my_init)
net[2].weight


TypeError: 'generator' object is not subscriptable